In [1]:
import pandas as pd
import numpy as np

from collections import defaultdict

from sklearn.model_selection import train_test_split

In [85]:
data_dir = '/lustre/groups/epigenereg01/workspace/projects/vale/mlm/'

In [41]:
#get human 3'UTR sequences

human_fasta = data_dir + 'fasta/Homo_sapiens_rna.fa'

dataset = defaultdict(str)

with open(human_fasta, 'r') as f:
    for line in f:
        if line.startswith('>'):
            seq_name = line[1:].rstrip()
        else:
            dataset[seq_name] += line.rstrip().upper()
            
dataset = pd.DataFrame(list(dataset.items()), columns=['seq_name','seq'])

In [42]:
chrom = dataset.seq_name.apply(lambda x:x.split('_')[-3])

In [43]:
#75%/25% train/test split, stratified by chromosome

train_df, test_df  = train_test_split(dataset, test_size=0.25, random_state=1,stratify=chrom)

In [44]:
train_df.to_csv(data_dir + '/motif_analysis/train.csv', index=None)
test_df.to_csv(data_dir + '/motif_analysis/test.csv', index=None)